In [1]:
import pandas as pd
from splycer.blocker import BlockDB
from splycer.record_set import RecordDB
from splycer.pairs_set import PairsDB
from splycer.feature_engineer import FeatureEngineer
import recordlinkage as rl
import pyodbc
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import pickle as pkl
from tqdm import tqdm

# Set up a database connection
import turbodbc
conn = turbodbc.connect('rec_db')

import os.path
basePath = r'R:\JoePriceResearch\record_linking\projects\deep_learning\paper_RR\CensusTree_2020\final'
trainPath = os.path.abspath(os.path.join(basePath, '2-split_train_test', 'train_1900_1910.csv'))
testPath = os.path.abspath(os.path.join(basePath, '2-split_train_test', 'test_1900_1910.csv'))

## Create the class for comparing features.

In [2]:
from recordlinkage.base import BaseCompareFeature

class eucledian_distance(BaseCompareFeature):
    def __init__(self, left_on, right_on):
        super(eucledian_distance, self).__init__(left_on, right_on)
        self.n = len(left_on)
    def _compute_vectorized(self,*args):
        s1 = args[:self.n]
        s2 = args[self.n:]
        return np.linalg.norm(np.array(s1)-np.array(s2),ord=2,axis=0)
    
class commonality_weight(BaseCompareFeature):
    def __init__(self,left_on,right_on):
        super(commonality_weight, self).__init__(left_on, right_on)
    def _compute_vectorized(self,s1,s2):
        return 1 / np.log1p((s1 + s2) / 2)
    
def get_compare_engine(drop=[]):
    exact_match_features = ['marstat','mbp','fbp','rel','first_nysiis','last_nysiis']
    exact_match_features = [feat for feat in exact_match_features if feat not in drop]
    c = rl.Compare() # declare comparison object
    if 'res' not in drop:
        c.geo('res_lat','res_lon','res_lat','res_lon',method = 'exp',scale=500)
    if 'bp' not in drop:
        c.geo('bp_lat','bp_lon','bp_lat','bp_lon', method = 'exp',scale=500)
    if 'first_jaro' not in drop:
        c.string('first','first',method = 'jarowinkler')
    if 'last_jaro' not in drop:
        c.string('last','last', method = 'jarowinkler')
    #c.string('first','first',method = 'qgram')
    #c.string('last','last', method = 'qgram')
    if 'birth_year' not in drop:
        c.numeric('birth_year','birth_year', method = 'lin', scale = 1, offset = 1)
    if 'immigration' not in drop:
        c.numeric('immigration','immigration', method = 'lin', scale = 1, offset = 1)
    
    vec_cols = [f'occ_vec{i}' for i in range(128)]
    if 'occ' not in drop:
        c.add(eucledian_distance(vec_cols,vec_cols))
    if 'comm_first' not in drop:
        c.add(commonality_weight('first_comm','first_comm'))
    if 'comm_last' not in drop:
        c.add(commonality_weight('last_comm','last_comm'))    
    for col in exact_match_features:
        c.exact(col,col)
    return c

## Load the training data

In [3]:
# Get the training set.
df = pd.read_csv(trainPath)

# Get the full data using SQL.
sql1900 = RecordDB('compiled_1900','ark1900','rec_db')
sql1910 = RecordDB('compiled_1910','ark1910','rec_db')
rec1900 = sql1900.get_records(df['ark1900'].drop_duplicates()).set_index('index')
rec1910 = sql1910.get_records(df['ark1910'].drop_duplicates()).set_index('index')

In [4]:
# Create the truth value.
pairs = pd.MultiIndex.from_arrays((df['ark1900'],df['ark1910']))
y = df['ark1910']==df['true_ark_1910']
y.value_counts(normalize=1)

False    0.891553
True     0.108447
dtype: float64

In [5]:
rec1900.index = rec1900.index_
rec1910.index = rec1910.index_

In [6]:
c = get_compare_engine(drop=['occ','first_nysiis','last_nysiis'])
X = c.compute(pairs,rec1900,rec1910)
X.columns=['res','bp','first_jaro','last_jaro','birth_year','immigration','first_comm',
           'last_comm','marstat','mbp','fbp','rel']

In [7]:
X.describe()

,res,bp,first_jaro,last_jaro,birth_year,immigration,first_comm,last_comm,marstat,mbp,fbp,rel
count,221390.000000,221390.000000,221390.000000,221390.000000,221390.000000,221390.000000,221375.000000,221365.000000,221390.000000,221390.000000,221390.000000,221390.000000
mean,0.583953,0.999968,0.928740,0.923849,0.614673,0.057932,0.077981,0.100075,0.749451,0.715050,0.698026,0.712679
std,0.330974,0.005623,0.165150,0.128159,0.413224,0.219378,0.018295,0.033077,0.433330,0.451391,0.459115,0.452514
min,0.000000,0.000000,0.455556,0.000000,0.000000,0.000000,0.069286,0.072734,0.000000,0.000000,0.000000,0.000000
25%,0.289866,1.000000,1.000000,0.880000,0.000000,0.000000,0.069342,0.080787,0.000000,0.000000,0.000000,0.000000
50%,0.657234,1.000000,1.000000,1.000000,0.500000,0.000000,0.073740,0.092054,1.000000,1.000000,1.000000,1.000000
75%,0.868035,1.000000,1.000000,1.000000,1.000000,0.000000,0.081179,0.107302,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.442695,1.091357,1.000000,1.000000,1.000000,1.000000


## Loading the test data

In [8]:
# Load in the test data.
val = pd.read_csv(testPath)
val.columns = ['ark1900','ark1910','true_ark1910']

val['truth'] = val['ark1910']==val['true_ark1910']
pairs = pd.MultiIndex.from_arrays((val['ark1900'],val['ark1910']))

recb = sql1910.get_records(val['ark1910'].drop_duplicates().tolist()).set_index('index')
reca = sql1900.get_records(val['ark1900'].drop_duplicates().tolist()).set_index('index')
reca.index=reca.index_
recb.index=recb.index_

test_X=c.compute(pairs,reca,recb)

test_y = val['truth']
test_X.columns=['res','bp','first_jaro','last_jaro','birth_year','immigration','first_comm',
           'last_comm','marstat','mbp','fbp','rel']

test_X.describe()

,res,bp,first_jaro,last_jaro,birth_year,immigration,first_comm,last_comm,marstat,mbp,fbp,rel
count,94882.000000,94882.000000,94882.000000,94882.000000,94882.000000,94882.000000,94866.000000,94870.000000,94882.000000,94882.000000,94882.000000,94882.000000
mean,0.584409,0.999937,0.928245,0.924030,0.612888,0.057919,0.078062,0.100241,0.749405,0.717576,0.700986,0.712664
std,0.330946,0.007952,0.165962,0.127848,0.413282,0.219466,0.021201,0.033681,0.433358,0.450181,0.457828,0.452522
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.069286,0.072734,0.000000,0.000000,0.000000,0.000000
25%,0.290949,1.000000,1.000000,0.880000,0.000000,0.000000,0.069342,0.080912,0.000000,0.000000,0.000000,0.000000
50%,0.657936,1.000000,1.000000,1.000000,0.500000,0.000000,0.073740,0.092193,1.000000,1.000000,1.000000,1.000000
75%,0.868460,1.000000,1.000000,1.000000,1.000000,0.000000,0.081179,0.107302,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.442695,1.091357,1.000000,1.000000,1.000000,1.000000


## Train using three algorithms

In [9]:
# Train on nearest centroid.
from sklearn.neighbors import NearestCentroid
model = NearestCentroid()
model.fit(X.fillna(X.mean()),y)

y_pred_val = model.predict(test_X.fillna(X.mean()))
y_pred = model.predict(X.fillna(X.mean()))

print(f'train_recall: {recall_score(y,y_pred)}')
print(f'train_precision: {precision_score(y,y_pred)}\n')
print(f'val recall: {recall_score(test_y,y_pred_val)}')
print(f'val precision: {precision_score(test_y,y_pred_val)}\n')
print(f'train_f1_score: {f1_score(y,y_pred)}')
print(f'test_f1_score: {f1_score(test_y, y_pred_val)}')

train_recall: 0.821317006122704
train_precision: 0.27007519208908004

val recall: 0.8162182644941266
val precision: 0.275324343324599

train_f1_score: 0.4064851270845788
test_f1_score: 0.4117562724014337


In [10]:
# Train using Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=300)
model.fit(X.fillna(X.mean()),y)

# Predict
y_pred_val = model.predict(test_X.fillna(X.mean()))
y_pred = model.predict(X.fillna(X.mean()))

# Print stats.
print(f'train_recall: {recall_score(y,y_pred)}')
print(f'train_precision: {precision_score(y,y_pred)}\n')
print(f'val recall: {recall_score(test_y,y_pred_val)}')
print(f'val precision: {precision_score(test_y,y_pred_val)}\n')
print(f'train_f1_score: {f1_score(y,y_pred)}')
print(f'test_f1_score: {f1_score(test_y, y_pred_val)}')

train_recall: 0.6537964929817985
train_precision: 0.8386941654199616

val recall: 0.650909435392194
val precision: 0.8455574698498647

train_f1_score: 0.7347922761849035
test_f1_score: 0.7355743496413661


In [11]:
# Train using XGB.
model = XGBClassifier()
model.fit(X,y)

y_pred_val = model.predict(test_X)
y_pred = model.predict(X)

print(f'train_recall: {recall_score(y,y_pred)}')
print(f'train_precision: {precision_score(y,y_pred)}\n')
print(f'test_recall: {recall_score(test_y,y_pred_val)}')
print(f'test_precision: {precision_score(test_y,y_pred_val)}\n')
print(f'train_f1_score: {f1_score(y,y_pred)}')
print(f'test_f1_score: {f1_score(test_y, y_pred_val)}')


train_recall: 0.8448914990212004
train_precision: 0.888679575922194

test_recall: 0.8226600985221675
test_precision: 0.8777036587831009

train_f1_score: 0.8662325184157148
test_f1_score: 0.8492909535452322


## Test micro-parameters for XGBoost

In [12]:
# Check the following micro parameters.
learning_rates=[.3,.4]
max_depth=[5,6]
alpha_vals = [0,0.5]
lambda_vals = [0,1]
n_jobs=16


for lr in learning_rates:
    for depth in max_depth:
        for alph in alpha_vals:
            for lam in lambda_vals:
                model = XGBClassifier(
                    learning_rate=lr, max_depth=depth, n_jobs=n_jobs,
                    reg_alpha=alph, reg_lambda=lam)
                model.fit(X,y)
                y_pred_val = model.predict(test_X)
                print(f1_score(test_y, y_pred_val), lr, depth, alph, lam)

0.8480516937536715 0.3 5 0 0
0.8460071397134333 0.3 5 0 1
0.8480516937536715 0.3 5 0.5 0
0.8480969365319782 0.3 5 0.5 1
0.8465634311953495 0.3 6 0 0
0.8492909535452322 0.3 6 0 1
0.8473353162026428 0.3 6 0.5 0
0.8487185745667563 0.3 6 0.5 1
0.84819206558337 0.4 5 0 0
0.8476898981989036 0.4 5 0 1
0.8483781006898575 0.4 5 0.5 0
0.8483041735900694 0.4 5 0.5 1
0.8487038031538349 0.4 6 0 0
0.8499706974018362 0.4 6 0 1
0.849599843841499 0.4 6 0.5 0
0.8508621953006692 0.4 6 0.5 1


## ReCreate and save our best model

In [13]:
model = XGBClassifier(learning_rate=0.4, max_depth=6, n_jobs=n_jobs,
                    reg_alpha=0.5, reg_lambda=1)
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.4, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0.5,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
y_pred_val = model.predict(test_X)
y_pred = model.predict(X)
tn, fp, fn, tp = confusion_matrix(test_y,y_pred_val).ravel()
tn, fp, fn, tp

(83120, 1206, 1847, 8709)

In [15]:
# Save the model.
import pickle
pickle.dump(model, open("model_1900_1910.dat", "wb"))

In [16]:
# Load the model
loaded_model = pickle.load(open("model_1900_1910.dat", "rb"))
loaded_model.predict(X)

array([False, False, False, ..., False, False, False])